In [11]:
%reload_ext tensorboard 
%tensorboard --logdir=content/tensorboard/cartpole

Reusing TensorBoard on port 6007 (pid 11842), started 0:00:04 ago. (Use '!kill 11842' to kill it.)

In [6]:
import muax
from muax import nn 

support_size = 10 
embedding_size = 8
discount = 0.99
num_actions = 4
full_support_size = int(support_size * 2 + 1)

repr_fn = nn._init_representation_func(nn.Representation, embedding_size)
pred_fn = nn._init_prediction_func(nn.Prediction, num_actions, full_support_size)
dy_fn = nn._init_dynamic_func(nn.Dynamic, embedding_size, num_actions, full_support_size)

tracer = muax.PNStep(10, discount, 0.5)
buffer = muax.TrajectoryReplayBuffer(500)

gradient_transform = muax.model.optimizer(init_value=0.02, peak_value=0.02, end_value=0.002, warmup_steps=5000, transition_steps=5000)

model = muax.MuZero(repr_fn, pred_fn, dy_fn, policy='muzero', discount=discount,
                    optimizer=gradient_transform, support_size=support_size)

model_path = muax.fit(model, 'CartPole-v1', 
                    max_episodes=1000,
                    max_training_steps=10000,
                    tracer=tracer,
                    buffer=buffer,
                    k_steps=10,
                    sample_per_trajectory=1,
                    num_trajectory=32,
                    tensorboard_dir='content/tensorboard/cartpole',
                    model_save_path='content/models/cartpole',
                    save_name='cartpole_model_params',
                    random_seed=0,
                    log_all_metrics=True)

buffer warm up stage...
start training...


[TrainMonitor|INFO] ep: 1,	T: 10,	G: 9,	avg_r: 1,	avg_G: 9,	t: 9,	dt: 1046.322ms,	v: 0.0394,	Rn: 4.87,	loss: 2.6,	training_step: 50,	test_G: 17.3
[TrainMonitor|INFO] ep: 2,	T: 23,	G: 12,	avg_r: 1,	avg_G: 10.5,	t: 12,	dt: 76.425ms,	v: 8.82,	Rn: 7.31,	loss: 1.97,	training_step: 100
[TrainMonitor|INFO] ep: 3,	T: 32,	G: 8,	avg_r: 1,	avg_G: 9.67,	t: 8,	dt: 222.135ms,	v: 8.51,	Rn: 4.4,	loss: 1.93,	training_step: 150
[TrainMonitor|INFO] ep: 4,	T: 41,	G: 8,	avg_r: 1,	avg_G: 9.25,	t: 8,	dt: 115.250ms,	v: 9.24,	Rn: 4.4,	loss: 1.92,	training_step: 200
[TrainMonitor|INFO] ep: 5,	T: 55,	G: 13,	avg_r: 1,	avg_G: 10,	t: 13,	dt: 70.764ms,	v: 10,	Rn: 8.26,	loss: 1.9,	training_step: 250
[TrainMonitor|INFO] ep: 6,	T: 64,	G: 8,	avg_r: 1,	avg_G: 9.67,	t: 8,	dt: 19.527ms,	v: 9.63,	Rn: 4.4,	loss: 1.9,	training_step: 300
[TrainMonitor|INFO] ep: 7,	T: 80,	G: 15,	avg_r: 1,	avg_G: 10.4,	t: 15,	dt: 10.569ms,	v: 10.5,	Rn: 9.91,	loss: 1.92,	training_step: 350
[TrainMonitor|INFO] ep: 8,	T: 89,	G: 8,	avg_r: 1,	avg_G: 

In [14]:
import gymnasium as gym
import jax
from muax.test import test

model = muax.MuZero(repr_fn, pred_fn, dy_fn, policy='muzero', discount=discount,
                    optimizer=gradient_transform, support_size=support_size)

model.load(model_path)

env_id = 'CartPole-v1'
test_env = gym.make(env_id, render_mode='rgb_array')
test_key = jax.random.PRNGKey(0)
test(model, test_env, test_key, num_simulations=50, num_test_episodes=100, random_seed=None)

130.53